# Summarization of Financial Data Using a Large Language Model (LLM)

This notebook shows model developers how to document a large language model (LLM) using the ValidMind Developer Framework. The use case is a summarization of financial news based on a dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. It shows you how to set up the ValidMind Developer Framework, initialize the client library, and load the dataset, followed by running the model validation tests provided by the framework to quickly generate documentation about the data and model.

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

This notebook requires an OpenAI API secret key to run. If you don't have one, visit [API keys](https://platform.openai.com/account/api-keys) on OpenAI's site to create a new key for yourself. Note that API usage charges may apply.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [1]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Foundation Model - Text Summarization`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace the code below with the code snippet from your project ##


import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)
  

2024-01-19 12:41:55,166 - INFO(validmind.api_client): Connected to ValidMind. Project: [Demo] Foundation Model  - Text Summarization - Initial Validation (clr4sqrfh0050uqy62007ncxt)


### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [3]:
vm.preview_template()

Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

### Helper functions

Let's define the following functions to help visualize datasets with long text fields:

In [4]:
import textwrap

from IPython.display import display, HTML
from tabulate import tabulate


def _format_cell_text(text, width=50):
    """Private function to format a cell's text."""
    return '\n'.join([textwrap.fill(line, width=width) for line in text.split('\n')])


def _format_dataframe_for_tabulate(df):
    """Private function to format the entire DataFrame for tabulation."""
    df_out = df.copy()

    # Format all string columns
    for column in df_out.columns:
        # Check if column is of type object (likely strings)
        if df_out[column].dtype == object:
            df_out[column] = df_out[column].apply(_format_cell_text)
    return df_out


def _dataframe_to_html_table(df):
    """Private function to convert a DataFrame to an HTML table."""
    headers = df.columns.tolist()
    table_data = df.values.tolist()
    return tabulate(table_data, headers=headers, tablefmt="html")


def display_formatted_dataframe(df, num_rows=None):
    """Primary function to format and display a DataFrame."""
    if num_rows is not None:
        df = df.head(num_rows)
    formatted_df = _format_dataframe_for_tabulate(df)
    html_table = _dataframe_to_html_table(formatted_df)
    display(HTML(html_table))

### Load the dataset

The CNN Dailymail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail (https://huggingface.co/datasets/cnn_dailymail). The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

In [5]:
from datasets import load_dataset

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')
train_df = cnn_dataset.data['train'].to_pandas()
val_df = cnn_dataset.data['validation'].to_pandas()
test_df = cnn_dataset.data['test'].to_pandas()
train_df = train_df[['article', 'highlights']]
train_df = train_df.head(20)

display_formatted_dataframe(train_df, num_rows=5)

### Get ready to run the analysis

Import the ValidMind `FoundationModel` and `Prompt` classes needed for the sentiment analysis later on:

In [6]:
from validmind.models import FoundationModel, Prompt

Check your access to the OpenAI API:

In [7]:
import os

import dotenv
dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")

In [8]:
from openai import OpenAI

model = OpenAI()

def call_model(prompt):
    return model.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message.content

Set the prompt guidelines for the sentiment analysis:

In [9]:
prompt_template = """
You are an AI with expertise in summarizing financial news.
Your task is to provide a concise summary of the specific news article provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the article.

Article to Summarize:

```
{article}
```

Please respond with a concise summary of the article's main points.
Ensure that your summary is based on the content of the article and not on external information or assumptions.
""".strip()

prompt_variables = ["article"]

In [10]:
df_test = train_df.head(10)

vm_test_ds = vm.init_dataset(
    dataset=df_test,
    text_column="article",
    target_column="highlights",
)

vm_model = FoundationModel(
    predict_fn=call_model,
    prompt=Prompt(
        template=prompt_template,
        variables=prompt_variables,
    ),
    test_ds=vm_test_ds,
)

2024-01-19 12:42:00,717 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-01-19 12:42:00,718 - INFO(validmind.models.foundation): Running predict() for `test_ds`... This may take a while


### Run model validation tests

It's possible to run a subset of tests on the documentation template by passing a `section` parameter to `run_documentation_tests()`. Let's run the tests that evaluate the model's overall performance (including summarization metrics), by selecting the "model development" section of the template:

In [12]:
config = {
    "rouge_metric": {
        "rouge_metrics": ["rouge-1", "rouge-2", "rouge-l"],
    },
}

summarization_results = vm.run_documentation_tests(
    section="model_development",
    inputs = {"model":vm_model},
    config=config,
)

2024-01-19 12:43:19,609 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'bias': (AttributeError) 'NoneType' object has no attribute 'chat'
2024-01-19 12:43:19,610 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'clarity': (AttributeError) 'NoneType' object has no attribute 'chat'
2024-01-19 12:43:19,610 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'conciseness': (AttributeError) 'NoneType' object has no attribute 'chat'
2024-01-19 12:43:19,611 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'delimitation': (AttributeError) 'NoneType' object has no attribute 'chat'
2024-01-19 12:43:19,611 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'negative_instruction': (AttributeError) 'NoneType' object has no attribute 'chat'
2024-01-19 12:43:19,612 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'specificity': (AttributeError) 'NoneType' object has no attribute 'chat'
Some weights of Roberta

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the prompt validation test results as part of your model documentation right in the ValidMind Platform UI: 

1. Log back into the [Platform UI](https://app.prod.validmind.ai) 

2. Go to **Documentation Projects** > **YOUR_DOCUMENTATION_PROJECT** > **Documentation**.

3. Expand **2. Data Preparation** or **3. Model Development** to review all test results.

What you can see now is a more easily consumable version of the prompt validation testing you just performed, along with other parts of your documentation project that still need to be completed. 

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).

